In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [3]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [4]:
filename = 'YZV_data.csv'
HV_data = pd.read_csv(path + filename)

In [5]:
set_index(HV_data)

In [6]:
tm_data = HV_data

In [7]:
HV_data = HV_data.drop(columns=['YZV'])

In [8]:
HV_data['YZV'] = tm_data['YZV']

In [9]:
HV_data

,gdp_growth,volume,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)",...,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,"('APARCH', 'studentst', 1)",YZV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,28984.929215,2.945453e+09,0.039601,0.038301,0.039601,0.038301,0.040044,0.039199,0.038388,0.038388,...,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.037128,0.282636
2000-02-03,28984.929215,2.935907e+09,0.037979,0.038355,0.037979,0.038355,0.041000,0.040608,0.038633,0.038633,...,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.038336,0.285768
2000-02-04,28984.929215,3.274647e+09,0.036791,0.037755,0.036791,0.037755,0.038214,0.040159,0.036990,0.036990,...,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.049778,0.279604
2000-02-07,28984.929215,3.413593e+09,0.046593,0.055072,0.046593,0.055072,0.047968,0.057512,0.046474,0.046474,...,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.065732,0.291652
2000-02-08,28984.929215,2.868803e+09,0.049131,0.041984,0.049131,0.041984,0.049560,0.042487,0.046711,0.046711,...,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.043373,0.293459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000000,1.609587e+08,0.000120,-0.000665,0.000120,-0.000665,0.000456,-0.000424,0.000121,0.000121,...,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,-0.004076,0.000035
2024-08-27,0.000000,9.767973e+07,-0.003889,-0.004798,-0.003889,-0.004798,-0.003539,-0.002953,-0.004368,-0.004368,...,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,-0.005604,-0.000063
2024-08-28,0.000000,2.498703e+08,0.000280,-0.001148,0.000280,-0.001148,0.000449,-0.000383,0.000483,0.000483,...,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,-0.001642,0.000235


# Split train, validation and test data

In [10]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [11]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [12]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [13]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## YZV

In [14]:
daily_train, daily_val, daily_test = split_data(HV_data, 'YZV')

In [15]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [16]:
HV_X_train, HV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
HV_X_val, HV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
HV_X_test, HV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [17]:
# HV_X_train = HV_X_train.reshape(HV_X_train.shape[0], win_sz, -1)
# HV_X_val = HV_X_val.reshape(HV_X_val.shape[0], win_sz, -1)
# HV_X_test = HV_X_test.reshape(HV_X_test.shape[0], win_sz, -1)

### LSTM

In [18]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │          14,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,221 (75.08 KB)

 Trainable params: 19,221 (75.08 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
lstm_history = lstm_garch_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.0125 - val_loss: 3.2476e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - loss: 9.4584e-04 - val_loss: 5.5924e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 7.0687e-04 - val_loss: 6.0192e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 8.2020e-04 - val_loss: 3.0908e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 6.5217e-04 - val_loss: 3.2783e-05
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 7.0044e-04 - val_loss: 4.1821e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 6.3259e-04 - val_loss: 2.8489e-05
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 7.2226e-04 - val_loss: 5.4211e-05
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 6.7189e-04 - val_loss: 2.7990e-05
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 7.1681e-04 - val_loss: 2.5488e-05
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step

In [21]:
# Predictions
HV_test_predictions = lstm_garch_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [22]:
mse = mean_squared_error(HV_y_test, HV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 0.00016572878335683084
RMSE: 0.012873569177070936
MAE: 0.012673681865779657
MAPE: 39117.104769392325%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
gru_model.summary()

In [ ]:
gru_history = gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - loss: 0.0632 - val_loss: 3.6607e-04
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 0.0018 - val_loss: 1.7478e-04
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - loss: 0.0013 - val_loss: 1.1597e-04
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 0.0013 - val_loss: 1.0096e-04
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0010 - val_loss: 1.8494e-04
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0010 - val_loss: 6.1036e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 8.8664e-04 - val_loss: 5.1956e-05
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 8.0989e-04 - val_loss: 2.9910e-05
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 7.3245e-04 - val_loss: 7.8300e-05
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 9.4268e-04 - val_loss: 4.4587e-05
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 8.1271e-04 

In [ ]:
# Predictions
HV_gru_predictions = gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 1.1274369545255398e-05
RMSE: 0.0033577327983708586
MAE: 0.0027728838272739697
MAPE: 7665.4623472202675%


### LSTM_GRU

In [ ]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_gru_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 25, 48)              │          14,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,229 (71.21 KB)

 Trainable params: 18,229 (71.21 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_gru_history = lstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - loss: 0.0250 - val_loss: 1.0653e-04
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 0.0013 - val_loss: 1.3177e-04
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 0.0010 - val_loss: 9.6100e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 9.8355e-04 - val_loss: 6.2223e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 7.6299e-04 - val_loss: 4.0089e-05
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 7.2777e-04 - val_loss: 9.8572e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 7.2141e-04 - val_loss: 2.4106e-05
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 6.0377e-04 - val_loss: 2.7317e-05
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 5.9715e-04 - val_loss: 3.2481e-05
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 8.5985e-04 - val_loss: 2.8602e-05
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss:

In [ ]:
# Predictions
HV_lstm_gru_predictions = lstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.000234427674218491
RMSE: 0.015311031128519432
MAE: 0.014800092278815524
MAPE: 45651.51560875367%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/Predictions/YZV/'

# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)

### BiLSTM

In [ ]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Bidirectional(LSTM(16)))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Dense(5))

bilstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
bilstm_model.summary()

In [ ]:
bilstm_history = bilstm_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - loss: 0.0177 - val_loss: 4.2208e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - loss: 8.8145e-04 - val_loss: 2.9963e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 8.0393e-04 - val_loss: 2.2512e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 56ms/step - loss: 8.0191e-04 - val_loss: 1.9186e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - loss: 7.3654e-04 - val_loss: 3.8109e-05
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 9.0843e-04 - val_loss: 2.5151e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - loss: 6.5407e-04 - val_loss: 1.7718e-05
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - loss: 6.7525e-04 - val_loss: 1.6765e-05
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 8.0135e-04 - val_loss: 1.0758e-05
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - loss: 6.9320e-04 - val_loss: 7.6103e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 55

In [ ]:
# Predictions
HV_bilstm_predictions = bilstm_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_predictions)
print(f'MAPE: {mape}%')

MSE: 9.323140031285594e-06
RMSE: 0.0030533817369083733
MAE: 0.002742895138710419
MAPE: 8144.59784525041%


### BiLSTM_GRU

In [ ]:
bilstm_gru_model = Sequential()
bilstm_gru_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(GRU(16))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(Dense(5))

bilstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_gru_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_3 (Bidirectional)      │ (None, 25, 96)              │          29,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_4 (GRU)                          │ (None, 16)                  │           5,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,509 (138.71 KB)

 Trainable params: 35,509 (138.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bilstm_gru_history = bilstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - loss: 0.0262 - val_loss: 3.0655e-04
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 0.0013 - val_loss: 5.2547e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - loss: 9.0531e-04 - val_loss: 9.8847e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - loss: 7.0157e-04 - val_loss: 1.4688e-04
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 7.1795e-04 - val_loss: 2.4832e-05
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 7.0189e-04 - val_loss: 2.7126e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - loss: 7.3906e-04 - val_loss: 2.9867e-05
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - loss: 7.5410e-04 - val_loss: 3.2828e-04
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 0.0014 - val_loss: 1.2020e-04
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - loss: 7.1896e-04 - val_loss: 2.6469e-05
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - lo

In [ ]:
# Predictions
HV_bilstm_gru_predictions = bilstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 7.748081829214437e-05
RMSE: 0.008802318915612202
MAE: 0.008620217827529914
MAPE: 26788.837920324146%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/YZV/'

# df_pred = pd.DataFrame(HV_test_predictions[:,0], columns=['lstm_pred'])
# filename = 'LSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_gru_predictions[:,0], columns=['gru_pred'])
# filename = 'GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_predictions[:,0], columns=['bilstm_pred'])
# filename = 'BiLSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_gru_predictions[:,0], columns=['bilstm_gru_pred'])
# filename = 'BiLSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)